# Задание 9.5. Модуль ML-6 (HW-03)

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. 

__КРИТЕРИИ ОЦЕНИВАНИЯ__:

__1 балл__ - Верно выделены три столбца-признака для обучения, выбранные RFE.

__1 балл__ - Верно выделены три столбца-признака для обучения, выбранные SelectKBest.

__3 балла__ - Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.

__3 балла__ - Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.

__2 балла__ - Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

Максимальное количество баллов за выполнение задания — __10__.

In [44]:
# Импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

# Загрузка и обработка данных

In [45]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [46]:
#Выберем только числовые признаки и удалим пропуски в данных
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

#Составляем матрицу наблюдений X и вектор ответов y
y = data['price']
x = data.drop(columns='price')

In [47]:
#Разделим данные на тестовые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Рекурсивное исключения признаков

In [48]:
#Выделим три столбца-признака для обучения, выбранные RFE
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)

#Занесем названия выбранных признаков в отдельную переменную 
rfe_features = selector.get_feature_names_out()

#Выведем назнавания признаков
print(rfe_features)

['year' 'cylinders' 'lat']


##  ВЫБОР ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [49]:
#Выделим три столбца-признака для обучения, выбранные SelectKBes
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)

#Занесем названия выбранных признаков в отдельную переменную 
kbest_features = selector.get_feature_names_out()

#Выведем назнавания признаков
print(kbest_features)

['year' 'cylinders' 'odometer']


## ОБУЧЕНИЕ РЕГРЕССИИ НА ПРИЗНАКАХ, ВЫБРАННЫХ RFE 

In [50]:
#Обучим модель на столбцах, выделенных RFE
model = LinearRegression()
model.fit(X_train[rfe_features], y_train)
y_predicted = model.predict(X_test[rfe_features])

#Рассчитаем метрики 
mae = mean_absolute_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print('MAE: %.3f' % mae)
print('R2 score: %.3f' % r2)


MAE: 5096.570
R2 score: 0.576


## ОБУЧЕНИЕ РЕГРЕССИИ НА ПРИЗНАКАХ, ВЫБРАННЫХ SELECTKBEST

In [51]:
#Обучим модель на столбцах, выделенных SelectKBest
model = LinearRegression()
model.fit(X_train[kbest_features], y_train)
y_predicted = model.predict(X_test[kbest_features])

#Рассчитаем метрики 
mae = mean_absolute_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

print('MAE: %.3f' % mae)
print('R2 score: %.3f' % r2)

MAE: 4708.946
R2 score: 0.602


## Выводы

Из предоставленных метрик видно, что второй метод - SelectKBest - показал лучший результат на тестовой выборке по обеим метрикам: MAE (Mean Absolute Error) и R2 score (коэффициент детерминации).

MAE (Mean Absolute Error):

- RFE: MAE = 5096.570
- SelectKBest: MAE = 4708.946

Меньшее значение MAE свидетельствует о лучшей точности модели. Таким образом, второй метод (SelectKBest) показывает более низкую среднюю абсолютную ошибку, что говорит о том, что модель, построенная с использованием SelectKBest, более точно предсказывает целевую переменную.

R2 score:

- RFE: R2 score = 0.576
- SelectKBest: R2 score = 0.602

R2 score измеряет объясненную дисперсию. Значение R2 score ближе к 1 указывает на лучшую объясненную вариацию целевой переменной моделью. Таким образом, второй метод (SelectKBest) с более высоким R2 score также показывает лучший результат в объяснении дисперсии.

Итак, основываясь на предоставленных метриках, можно заключить, что метод SelectKBest превосходит RFE для данного набора данных и задачи.